# Homework 8: I/O and data structures practice

## Goal

In this assignment we will practice working with files, more practice with dictionaries, and performing some basic geometric computations. Recall earlier in the course I walked through an example of processing retail data where we
had data about a set of products in a store, and the set of baskets that people bought.  We will work
with that and extend it a little for this assignment.  Our working data will be the following:

- _Product inventory_: This will be a table of products where each row corresponds to one product, and each column represents a property of the objects.  Specifically, our product inventory table will have five columns: a unique product ID, a text description of the product, a unit price in dollars, an X coordinate, and a Y coordinate.

- _Sales data_: This will be a table of records from the point of sale system.  Each row will correspond to an item in a basket.  The columns will be the basket ID, the product ID, product quantity, and the product pick up order.  The product pick up order for a basket containing $n$ items will range from 1 to $n$ and corresponds to the order that the customer picked up the products as they went through the store.

We will also be provided additional parameters about the store.  Specifically, we will be given the X,Y
coordinates of the entry door, the checkout stand, and the exit door.  We are going to assume that this
store has only one checkout line and all customers are good at following signs and always use the appropriate door for entry and exit.  We are also going to assume that customers have the magical ability to teleport through shelves so that they can take the shortest path from one product to the next, allowing us to avoid worrying about complex calculations of the distance between any two products.

The coordinates of the entrance, exit, and checkout are:

In [167]:
import math

In [168]:
store_entrance = (10,0)
store_exit = (90,0)
store_checkout = (50,10)

## Sample I/O code

To get you started, here is code that reads in the inventory data from the CSV file and produces a dictionary of dictionaries.  The outer dictionary maps product IDs as strings to the inner dictionary, and the inner dictionary for each product maps an attribute (e.g., 'unit_price') to its value.  Numerical values are stored as floating point numbers, _not strings_.

In [64]:
import csv

In [65]:
def read_inventory(filename):
    # initialize empty dict
    inventory={}
    
    # open the given file and name the file handle f
    with open(filename) as f:
        # create a CSV reader object from the file
        reader = csv.reader(f)
        
        # advance the reader to skip the first header line
        next(reader)
        
        # for each row in the CSV file, create the appropriate
        # entry in the inventory.  this includes converting the
        # strings for price, x, and y into floats so we can do
        # arithmetic with them later.
        for row in reader:
            inventory[row[0]] = { 'desc':row[1],
                                  'unit_price':float(row[2]),
                                  'x':float(row[3]),
                                  'y':float(row[4]) }
            
        return inventory

In [66]:
inventory = read_inventory('inventory.csv')

Test it: what is the price of product ID 4?

In [67]:
inventory['4']['unit_price']

0.39

In [83]:
inventory['2']

{'desc': 'apple', 'unit_price': 0.49, 'x': 4.0, 'y': 70.0}

## Part 1: read the basket data

Complete the following function to return a dictionary mapping basket ID to some data structure of your choice to represent the basket contents.

In [84]:
def read_baskets(filename):
    # initialize the empty dict
    baskets = {}
    
    # open the given file and name the file handle f
    with open(filename, 'r') as f:
        reader = csv.reader(f)
        next(reader)
        
        items = []
        basket_ids = []
        
        for row in reader:
            items.append(row)
            if row[0] not in basket_ids:
                basket_ids.append(row[0])
                
        for basket_id in basket_ids:
            baskets[basket_id] = []
        
        for item in items: 
            baskets[item[0]].append({
                'product_id': item[1],
                'quantity': float(item[2]),
                'pickup_order': float(item[3])
            })

        return baskets

In [85]:
baskets = read_baskets('baskets.csv')

In [86]:
baskets

{'1': [{'pickup_order': 1.0, 'product_id': '1', 'quantity': 2.0},
  {'pickup_order': 2.0, 'product_id': '3', 'quantity': 1.0}],
 '10': [{'pickup_order': 1.0, 'product_id': '1', 'quantity': 1.0},
  {'pickup_order': 2.0, 'product_id': '3', 'quantity': 2.0},
  {'pickup_order': 4.0, 'product_id': '5', 'quantity': 2.0},
  {'pickup_order': 3.0, 'product_id': '7', 'quantity': 1.0}],
 '2': [{'pickup_order': 1.0, 'product_id': '1', 'quantity': 3.0},
  {'pickup_order': 2.0, 'product_id': '4', 'quantity': 1.0},
  {'pickup_order': 3.0, 'product_id': '3', 'quantity': 1.0},
  {'pickup_order': 4.0, 'product_id': '11', 'quantity': 3.0}],
 '3': [{'pickup_order': 3.0, 'product_id': '13', 'quantity': 1.0},
  {'pickup_order': 2.0, 'product_id': '5', 'quantity': 2.0},
  {'pickup_order': 1.0, 'product_id': '6', 'quantity': 2.0}],
 '4': [{'pickup_order': 1.0, 'product_id': '7', 'quantity': 1.0},
  {'pickup_order': 3.0, 'product_id': '3', 'quantity': 1.0},
  {'pickup_order': 5.0, 'product_id': '12', 'quantity

In [96]:
baskets['1'][0]['product_id']

'1'

In [95]:
baskets['1'][1]['product_id']

'3'

In [164]:
# need to print every item in the 'baskets' dictionary
for i in baskets:
    total_price = 0
    print(" ")
    print("Basket:", i)
    basket_number = str(i)
    for n in baskets[basket_number]:
        # print(n)
        product = str(n['product_id'])
        print( inventory[product]['desc'])
        total_price += inventory[product]['unit_price'] * n['quantity']
    print("Total basket cost:", total_price)

 
Basket: 1
banana
cucumber
Total basket cost: 1.97
 
Basket: 2
banana
onion
cucumber
cheddar cheese
Total basket cost: 17.85
 
Basket: 3
coffee
potato chips
coca cola classic 6-pack
Total basket cost: 21.950000000000003
 
Basket: 4
whole milk
cucumber
salsa
ground beef 1lb
whole wheat fig bars
taco shells
Total basket cost: 29.43
 
Basket: 5
banana
apple
cucumber
onion
Total basket cost: 18.88
 
Basket: 6
onion
cucumber
apple
banana
Total basket cost: 4.23
 
Basket: 7
coffee
taco shells
Total basket cost: 13.49
 
Basket: 8
whole milk
Total basket cost: 1.99
 
Basket: 9
whole milk
whole wheat fig bars
coca cola classic 6-pack
onion
salsa
Total basket cost: 27.5
 
Basket: 10
banana
cucumber
potato chips
whole milk
Total basket cost: 8.44


## Part 2: Calculate the path length of each customer

Given the basket data and inventory data, write a function that calculates the distance traveled by a customer through the store.  Their trip should go entry -> first product -> second product -> ... -> checkout -> exit.  You should assume that the customer takes a straight path from each point to the next.

In [222]:
def dist(a, b):                                 # Distance to points passed to function
    return math.hypot(b[0] - a[0], b[1] - a[1])

def customer_trip(inventory, baskets, basket_id):
                                                # pre-defined: store_entrance, store_exit, store_checkout
    item_locations = []                         # init empty list of item locations
    
    total_distance = 0                          # init dist length for basket
    basket_number = str(basket_id)              # convert basket number is a string
    
    for n in baskets[basket_number]:            # gather products listing and add item location to list
        item = str( inventory[ n['product_id'] ]['desc'] )
        item_location_x = inventory[ n['product_id'] ]['x']
        item_location_y = inventory[ n['product_id'] ]['y']
        item_locations.append( (item_location_x, item_location_y))
 
    print('List of item locations:', item_locations)                   # print the list of all item locations
    
    total_distance += dist(store_entrance, item_locations[0])          # store_entrance --> first item 
    
    for i in range(1, len(item_locations)):
        total_distance += dist(item_locations[1], item_locations[i-1]) # first item --> second item --> n items
        
    total_distance += dist(item_locations[-1], store_checkout)         # last item --> store_checkout
    
    total_distance += dist(store_checkout, store_exit)                 # store_checkout --> store_exit
    
    return total_distance

In [229]:
for i in baskets:
    distance = customer_trip(inventory, baskets, i)
    print('Total distance for basket is',distance)

List of item locations: [(4.0, 80.0), (30.0, 75.0)]
Total distance for basket is 215.93949787012758
List of item locations: [(4.0, 80.0), (30.0, 65.0), (30.0, 75.0), (75.0, 30.0)]
Total distance for basket is 193.48802396347443
List of item locations: [(96.0, 70.0), (30.0, 55.0), (25.0, 30.0)]
Total distance for basket is 251.81709268444462
List of item locations: [(50.0, 90.0), (30.0, 75.0), (96.0, 80.0), (60.0, 55.0), (60.0, 75.0), (60.0, 65.0)]
Total distance for basket is 352.86596943976394
List of item locations: [(4.0, 80.0), (4.0, 70.0), (30.0, 75.0), (30.0, 65.0)]
Total distance for basket is 216.4556448800485
List of item locations: [(30.0, 65.0), (30.0, 75.0), (4.0, 70.0), (4.0, 80.0)]
Total distance for basket is 229.47637975508894
List of item locations: [(96.0, 70.0), (60.0, 65.0)]
Total distance for basket is 244.36564958947673
List of item locations: [(50.0, 90.0)]
Total distance for basket is 219.71963427413766
List of item locations: [(50.0, 90.0), (60.0, 75.0), (25.0,

## Part 3: Calculate the total price for each basket

Given the basket and inventory data, write a function that calculates the total cost of a basket.

In [146]:
def basket_total(inventory, baskets, basket_id):
    
    total_price = 0
    basket_number = str(basket_id)
    
    for n in baskets[basket_number]:
        # print(n)
        product = str(n['product_id'])
        total_price += inventory[product]['unit_price'] * n['quantity']

    print("Total basket cost:", total_price)
    return total_price

In [149]:
basket_total(inventory, baskets, '1')

Total basket cost: 1.97


1.97

## Part 4: Calculate the price per unit of distance traveled for all baskets

For each basket we have a distance traveled and a total price.  Write a function that returns a dictionary mapping the basket ID to the price per unit distance travelled.

In [10]:
def price_distance_calculate(inventory, baskets):
    # fill me in
    pass

## Part 5: EXTRA CREDIT.  

### Calculate the difference between the length of the path each customer took versus the shortest path they could have taken.

Each customer may have travelled the store inefficiently.  We would like to know the excess distance each customer covered versus what they could have done had they planned their trip more carefully.  Write a function that calculates the shortest path that a customer could have taken.

In [12]:
def customer_shortest_trip(inventory, baskets, basket_id):
    # fill me in
    pass